In [ ]:
#%%capture
print('NOTE: Intentionally crashing session to use the newly installed library.\n')
!pip uninstall -y pyarrow
!pip install ray[debug]==0.7.5
# A hack to force the runtime to restart, needed to include the above dependencies.
import os
os._exit(0)

NOTE: Intentionally crashing session to use the newly installed library.



In [3]:
%%capture
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [4]:
pip install google.colab


  Using cached google_colab-1.0.0-py2.py3-none-any.whl
  Using cached portpicker-1.2.0-py3-none-any.whl
  Using cached ipython-5.5.0-py3-none-any.whl (758 kB)
  Using cached ipykernel-4.6.1-py3-none-any.whl (104 kB)
  Using cached notebook-5.2.2-py2.py3-none-any.whl (8.0 MB)
  Using cached requests-2.21.0-py2.py3-none-any.whl (57 kB)
  Using cached pandas-0.24.2-cp38-cp38-win_amd64.whl
  Using cached google_auth-1.4.2-py2.py3-none-any.whl (64 kB)
  Using cached simplegeneric-0.8.1-py3-none-any.whl
  Using cached prompt_toolkit-1.0.18-py3-none-any.whl (245 kB)
  Using cached urllib3-1.24.3-py2.py3-none-any.whl (118 kB)
  Attempting uninstall: prompt-toolkit
    Found existing installation: prompt-toolkit 3.0.8
    Uninstalling prompt-toolkit-3.0.8:
      Successfully uninstalled prompt-toolkit-3.0.8
  Attempting uninstall: ipython
    Found existing installation: ipython 7.19.0
    Uninstalling ipython-7.19.0:
      Successfully uninstalled ipython-7.19.0
  Attempting uninstall: urllib3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorboard 2.4.0 requires google-auth<2,>=1.6.3, but you have google-auth 1.4.2 which is incompatible.
mxnet 1.6.0 requires numpy<1.17.0,>=1.8.2, but you have numpy 1.18.4 which is incompatible.
mxnet 1.6.0 requires requests<2.19.0,>=2.18.4, but you have requests 2.21.0 which is incompatible.
ipdb 0.13.6 requires ipython>=7.17.0; python_version > "3.6", but you have ipython 5.5.0 which is incompatible.
google-api-core 1.26.1 requires google-auth<2.0dev,>=1.21.1, but you have google-auth 1.4.2 which is incompatible.
google-api-core 1.26.1 requires six>=1.13.0, but you have six 1.12.0 which is incompatible.
gluonts 0.6.0 requires pandas>=1.0, but you have pandas 0.24.2 which is incompatible.
gluonts 0.6.0 requires pydantic<1.7,~=1.1, but you have pydantic 1.7.3 which is incompatible.
botocore 1.20.24 requires urlli

In [1]:
%%capture
!pip install python-Levenshtein
!pip install redis
!pip install -U ray
!pip install ray[debug]==0.7.5
!pip install ray[rllib]  # also recommended: ray[debug]
!pip uninstall -y pyarrow
!pip install unicodedata2



## Import

In [2]:
import os
os.chdir("/content/drive/My Drive/Knowledge Extraction/szhang37_code/KG_RL")

FileNotFoundError: [WinError 3] The system cannot find the path specified: '/content/drive/My Drive/Knowledge Extraction/szhang37_code/KG_RL'

In [ ]:
%tensorflow_version 1.x

In [ ]:
import tensorflow as tf
import pickle
import random
import pandas as pd 
import numpy as np
import json
import time

from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import unicodedata
from functools import reduce
#### Import ray related package
import ray
from ray.rllib.models import ModelCatalog
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.models.tf.fcnet_v2 import FullyConnectedNetwork

from ray.rllib.agents.ppo import PPOTrainer, DEFAULT_CONFIG
from ray.rllib.models.tf.misc import normc_initializer
from ray.rllib.models.tf.tf_modelv2 import TFModelV2
from ray.rllib.agents.dqn.distributional_q_model import DistributionalQModel
from ray.rllib.models.tf.visionnet_v2 import VisionNetwork as MyVisionNetwork
from ray.tune.logger import pretty_print
from ray.rllib.utils import try_import_tf
from ray.tune import grid_search
from ray.rllib.models import ModelCatalog
from ray.tune import Trainable
from ray.tune.logger import pretty_print
from ray.tune import run as run_tune
from ray.tune.registry import register_env
import gym
from gym import spaces
from gym.spaces import Discrete, Box
from ray import tune
from ray.rllib.agents.dqn.dqn import DQNTrainer, DEFAULT_CONFIG
### import self-defined function
import similarity_metrics
from utility_function import *
### import Environment
from Environment import KGRLEnv
### Import Customized DQN
from PolicyDQN import *
tf.__version__

## Data preparation

This section is used to obtain predicted data.

Skip this section when saved predicted data is provided.



### Bert

distance training for bert

In [ ]:
import os
import json
import pickle
########################################################
## convert the distance data into squad-like dataset ###
########################################################

def convert_distance_datasquad(file_name_input = 'sougou.json', file_name_output = None):
  
  data = {}
  data['data'] = []
  f = open(file_name_input, "r")
  lines = f.readlines()
  for i in range(len(lines)):
    line = lines[i]
    oneline = json.loads(line)
    answer = oneline['answer']
    passages = oneline['passages']
    question = oneline['question']
    for j in range(len(passages)):
      start_id = passages[j]['passage_text'].find(answer)
      if start_id == -1: continue
      entry = {}
      entry["paragraphs"] = [{
        'context' : passages[j]['passage_text'],
        'qas' : [{'question' : question,
                  'id' : "Trainbaidu_" + str(i) + "_context_" + str(j),
                  'answers' : [{'answer_start': start_id, 
                                'text': answer}]  }]
      }]
      data['data'].append(entry)
      break 

  with open(file_name_output, 'w') as outfile:
      json.dump(data, outfile)
  return data

distance_data_dir = "./distance_data/"
data = convert_distance_datasquad(file_name_input = distance_data_dir + 'sougou.json', file_name_output = distance_data_dir + 'sougou_squad.json')


In [ ]:
#####################################
## covert our data into squad-like ##
#####################################

def convert_squad(test_data, file_name = 'test_baidu_good.json'):
  

  data = {}
  data['data'] = []
  for i in range(len(test_data)):
    answer = test_data[i]['o']
    passages = test_data[i]['corpus']
    question = test_data[i]['s'] + "的" + test_data[i]['p'] + "是什么？"
    for j in range(len(passages)):
      start_id = None
      entry = {}
      entry["paragraphs"] = [{
        'context' : passages[j],
        'qas' : [{'question' : question,
                  'id' : "Trainbaidu_" + str(i) + "_context_" + str(j),
                  'answers' : [{'answer_start': start_id, 
                                'text': answer}]  }]
      }]
      data['data'].append(entry)
      #print(i)

        #break ## 每个问题有一个文本就好了。 (不然太太太多了？)


  with open(file_name, 'w') as outfile:
      json.dump(data, outfile)

train_data = pickle.load(open("./preprocessed_data/train_data.pkl", "rb" ))
test_data = pickle.load(open("./preprocessed_data/test_data.pkl", "rb" ))
store_dir = "./preprocessed_data/"
convert_squad(test_data, file_name = store_dir + 'test_data_squad.json')
convert_squad(train_data, file_name = store_dir + 'train_data_squad.json')


In [ ]:
## ref https://github.com/google-research/bert
#####################################################
## use bert to fine tune and predict on our data   ##
#####################################################

!python run_squad.py  \
    --vocab_file={/chinese_L-12_H-768_A-12/vocab.txt} \
    --bert_config_file={/chinese_L-12_H-768_A-12/bert_config.json} \
    --init_checkpoint={/chinese_L-12_H-768_A-12/bert_model.ckpt} \
    --do_train = True \
    --train_file={path to sougou.json} \
    --do_predict= False \
    --predict_file= {path to our data (test_data_squad/train_data_squad.json)} \
    --train_batch_size=12 --num_train_epochs=2.0 \
    --max_seq_length=384 \
    --doc_stride=128 \
    --learning_rate=3e-5 \
    --save_checkpoints_steps=1000 \
    --output_dir=./output/distant_supervision_full \
    --do_lower_case=True \
    --use_tpu=False





In [ ]:
############################
## pickle the bert output ##
############################

def pickle_answer(input_data, pred_file = "dev_predictions_test.json",\
                  pred_file_nbest = "dev_nbest_predictions_test.json",\
                  file_name_output = "pred_train_bert.pkl", saving_path = "./preprocessed_data/"):
  
  
  with open(pred_file, "r") as read_file:
      dev_data = json.load(read_file)

  with open(pred_file_nbest, "r") as read_file:
      dev_data_nbest = json.load(read_file)

  pred_output = {}
  for i in range(len(input_data)):
    tmp = []
    for j in range(len(input_data[i]['corpus'])):
      dev_id = 'Trainbaidu_%d_context_%d' %(i, j)
      try:
        tmp.append( [ [dev_data[dev_id]], [dev_data_nbest[dev_id][0]['probability']] ])
      except:
        tmp.append( [ ["ERROR"], [0] ])
    pred_output[input_data[i]['id']] = tmp
  with open(saving_path + file_name_output, 'wb') as f:
    pickle.dump(pred_output, f)
  
saving_path = "./preprocessed_data/"
bert_output_path = "./bert_output/"

train_data = pickle.load(open("./preprocessed_data/train_data.pkl", "rb" ))
test_data = pickle.load(open("./preprocessed_data/test_data.pkl", "rb" ))

pickle_answer(input_data = test_data, pred_file = bert_output_path + "dev_predictions_test.json",\
              pred_file_nbest = bert_output_path + "dev_nbest_predictions_test.json",\
              file_name_output = "pred_test_bert.pkl", saving_path = saving_path)

pickle_answer(input_data = train_data, pred_file = bert_output_path + "dev_predictions_train.json",\
              pred_file_nbest = bert_output_path + "dev_nbest_predictions_train.json",\
              file_name_output = "pred_train_bert.pkl", saving_path = saving_path)



### QANet

In [ ]:
# load pre-trained QANet
os.chdir("./qanet_model")
import qanet as net
from databunch import *
LoadModel(net)

Using TensorFlow backend.


qanet_mixall_alter0.h5


In [ ]:
def find_s_list(data, train = True):
  if train:
    s_list = set()
    for i in range(len(data)):
      s_list.add(data[i]['s'])
    s_list = list(s_list)
  else:
    s_list = set()
    for i in range(len(data)):
      s_list.add(data[i][0])
    s_list = list(s_list)
  return s_list

def GetAnswerListNew2(db_test, y_pred_start, y_pred_end, listans=True, addscore=[]):
    def GetAnswer(i, start, end):
        return ''.join(db_test.contextRaw[i][start:end + 1])
    answer = []
    predAnswerList = []
    for ii in range(db_test.numQuestions):
        cs, cspos = {}, {}
        for i in range(db_test.startEnd[ii][0], db_test.startEnd[ii][1] + 1):
            thisMax = [-1e+5] ;
            canswer = ["NaN**"] 
            for j1 in range(maxPLen):
                for j2 in range(j1, min(maxPLen, j1 + 8)):
                    score = y_pred_start[i][j1] * y_pred_end[i][j2]
                    if score > min(thisMax) or canswer == "NaN**":
                        temp = GetAnswer(i, j1, j2)
                        if temp in db_test.questionRaw[ii]: continue
                        canswer[np.argmin(thisMax)] = temp
                        thisMax[np.argmin(thisMax)] = score                        
                        mj1, mj2 = j1, j2
            # one passage
            if listans:
                tlen = len(db_test.contextRaw[i])
                if mj2 + 1 < tlen and db_test.contextRaw[i][mj2 + 1] == '、':
                    jj2 = mj2 + 1
                    while jj2 < tlen:
                        token = db_test.contextRaw[i][jj2];
                        jj2 += 1
                        #print(token)
                        if token[0] in '，。,. 被是': break
                        canswer += token;
                        mj2 += 1
            answer.append([canswer, thisMax])
    return answer

def GetCandidateAnswer(zz, query):
    ret = {'answer': '@NULL@', 'query': query, 'query_id': '0'}
    passages = []
    for text in zz:
        z = {'url': '', 'passage_text': text}
        passages.append(z)
    ret['passages'] = passages
    db = DataBunch(None, False, onejson=ret)
    X, Y = db.GetData()
    y_pred_start, y_pred_end = net.mm.predict(X, batch_size=128)  # ;print(y_pred_start,y_pred_end)
    corpus_addscore = [0.5, 0.5]
    isanslist = False
    ret = GetAnswerListNew2(db, y_pred_start, y_pred_end, isanslist, addscore=corpus_addscore)
    # print(ret)
    return ret
  

In [ ]:
os.chdir("/content/drive/My Drive/Knowledge Extraction/szhang37_code/KG_RL")
train_data = pickle.load(open("./preprocessed_data/train_data.pkl", "rb" ))
test_data = pickle.load(open("./preprocessed_data/test_data.pkl", "rb" ))
test_data_types = [test_data[i]['type'] for i in range(len(test_data))]

In [ ]:
train_pred = {}
for i in range(len(train_data)):
  key = 'Train_'+ str(i)
  entry = train_data[i]
  text_list = entry['corpus']
  query = entry['s'] + ' ' + entry['p']
  answer_list = GetCandidateAnswer(text_list, query)
  train_pred[key] = answer_list

Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
Loading model cost 0.741 seconds.
Prefix dict has been built successfully.


In [ ]:
test_pred = {}
for i in range(len(test_data)):
  key = 'Test_'+ str(i)
  entry = test_data[i]
  text_list = entry['corpus']
  query = entry['s'] + ' ' + entry['p']
  answer_list = GetCandidateAnswer(text_list, query)
  test_pred[key] = answer_list

In [ ]:
saving_path = "./preprocessed_data/"
with open(saving_path+"pred_train_qanet.pkl", 'wb') as f:
  pickle.dump(train_pred, f)
with open(saving_path+"pred_test_qanet.pkl", 'wb') as f:
  pickle.dump(test_pred, f)